In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
#!pip install tsfresh

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import numpy as np
import pandas as pd

#from tsfresh.transformers import RelevantFeatureAugmenter

import xgboost as xgb

from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_absolute_error as mae, make_scorer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

import statsmodels.api as sm
import statsmodels.formula.api as smf

sns.set(color_codes=True)
%matplotlib inline

In [ ]:
%cd /content/gdrive/Shared\ drives/DS4A_Colab

Steps for this jupyter notebook:

- NaN's, z-score
- transform (lags)
- recursive feature elimination (random forest)


In [ ]:
df = pd.read_csv('Data/Copia de santa_marta_consolidado_2008_2020.csv')
df['FECHA'] = pd.to_datetime(df['FECHA'])
df.drop(columns=['COD_DPTO','COD_MUNICIPIO','POBLACION'], inplace=True)

In [ ]:
df.fillna(method='ffill', inplace=True)

In [ ]:
df.set_index('FECHA', inplace=True)

In [ ]:
df_features = df[df.columns[:-3]].iloc[:-9]
df_target = df[df.columns[-3:]].iloc[:-9]
#The last entries are used as a final test set.

In [ ]:
df.head()

,ANO,SEMANA,PRECIPITATION,DRY_DAYS,TEMPERATURE_AMAX,REL_HUMIDITY_AMAX,TEMPERATURE_AMIN,REL_HUMIDITY_AMIN,TEMPERATURE_MEAN,TEMPERATURE_RANGE,REL_HUMIDITY_MEAN,MEI,ONI,SST1+2,SST3,SST4,SST3.4,TNI,DENGUE,DENGUE GRAVE,MORTALIDAD POR DENGUE
FECHA,,,,,,,,,,,,,,,,,,,,,
2007-12-30,2008,1,0.0,7,32.271698,75.147687,23.086833,46.720755,27.768733,9.136353,60.832122,-1.120000,-1.592857,23.048571,23.975714,26.814286,24.784286,1.554286,8,1,0
2008-01-06,2008,2,0.0,7,32.271698,72.790566,23.120377,46.720755,27.759330,9.151321,60.171473,-1.080000,-1.590000,23.860000,24.130000,26.620000,24.710000,1.836000,9,0,0
2008-01-13,2008,3,0.0,7,32.271698,72.790566,23.120377,46.720755,27.759330,9.151321,60.171473,-1.080000,-1.590000,23.860000,24.130000,26.620000,24.710000,1.836000,6,0,0
2008-01-20,2008,4,0.0,7,32.271698,72.790566,23.120377,46.720755,27.759330,9.151321,60.171473,-1.080000,-1.590000,23.860000,24.130000,26.620000,24.710000,1.836000,9,1,0
2008-01-27,2008,5,0.0,7,32.300000,72.790566,20.800000,44.000000,27.064088,7.600377,59.111890,-1.134286,-1.541429,24.562857,24.392857,26.565714,24.744286,1.998286,8,1,0


In [ ]:
scaler = StandardScaler()
scaler.fit(df_features)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
std_df_features = pd.DataFrame(scaler.transform(df_features)).set_index(df.iloc[:-9].index)
std_df_features = pd.concat([std_df_features, df_target['DENGUE']], axis=1)
std_df_features.columns = list(df_features.columns) + ['DENGUE']

In [ ]:
std_df_features.head()

,ANO,SEMANA,PRECIPITATION,DRY_DAYS,TEMPERATURE_AMAX,REL_HUMIDITY_AMAX,TEMPERATURE_AMIN,REL_HUMIDITY_AMIN,TEMPERATURE_MEAN,TEMPERATURE_RANGE,REL_HUMIDITY_MEAN,MEI,ONI,SST1+2,SST3,SST4,SST3.4,TNI,DENGUE
FECHA,,,,,,,,,,,,,,,,,,,
2007-12-30,-1.589155,-1.670965,-0.455648,0.853025,-1.322594,-1.542354,-0.935615,-0.504748,-1.055321,1.890289,-1.281835,-0.937968,-1.840244,-0.117165,-1.586529,-2.288594,-2.208234,1.619153,8
2008-01-06,-1.589155,-1.605010,-0.455648,0.853025,-1.322594,-1.836476,-0.911139,-0.504748,-1.064943,1.906992,-1.374210,-0.895766,-1.836989,0.231730,-1.466517,-2.524909,-2.278717,1.818744,9
2008-01-13,-1.589155,-1.539055,-0.455648,0.853025,-1.322594,-1.836476,-0.911139,-0.504748,-1.064943,1.906992,-1.374210,-0.895766,-1.836989,0.231730,-1.466517,-2.524909,-2.278717,1.818744,6
2008-01-20,-1.589155,-1.473100,-0.455648,0.853025,-1.322594,-1.836476,-0.911139,-0.504748,-1.064943,1.906992,-1.374210,-0.895766,-1.836989,0.231730,-1.466517,-2.524909,-2.278717,1.818744,9
2008-01-27,-1.589155,-1.407145,-0.455648,0.853025,-1.301945,-1.836476,-2.604185,-0.919959,-1.776327,0.176349,-1.522366,-0.953040,-1.781655,0.533941,-1.262053,-2.590939,-2.246186,1.933722,8


In [ ]:
def lagged_df(df, lag_size):
  dfs = list()
  for i in range(lag_size):
    df_lag = df.shift(i+1)
    df_lag.columns = [col+'_lag_'+str(i+1) for col in df.columns]
    dfs.append(df_lag)
  return pd.concat([df]+dfs, axis=1)

In [ ]:
std_df_features_lag = lagged_df(std_df_features, 12)

In [ ]:
#rfecv = RFECV(estimator=RandomForestRegressor(n_estimators=500, random_state=1), step=1, cv=TimeSeriesSplit(2))
rfecv = RFECV(estimator=xgb.XGBRegressor(objective="reg:squarederror", random_state=1), step=1,
              cv=TimeSeriesSplit(3), min_features_to_select=10)

In [ ]:
best_cols = dict()
for i in range(8):
  rfecv.fit(std_df_features_lag.iloc[12:-i-1], df_target['DENGUE'].shift(-i-1).iloc[12:-i-1])
  best_cols[str(i+1)+'_weeks_ahead'] = [std_df_features_lag.columns[i] for i in range(len(rfecv.support_)) if rfecv.support_[i]][:10]

In [ ]:
best_features = pd.DataFrame(best_cols)

In [ ]:
best_features

,1_weeks_ahead,2_weeks_ahead,3_weeks_ahead,4_weeks_ahead,5_weeks_ahead,6_weeks_ahead,7_weeks_ahead,8_weeks_ahead
0,ANO,ANO,ANO,ANO,ANO,ANO,ANO,ANO
1,SEMANA,SST4,SEMANA,REL_HUMIDITY_MEAN,PRECIPITATION,PRECIPITATION,SEMANA,SST1+2
2,TEMPERATURE_AMAX,DENGUE,REL_HUMIDITY_AMAX,ONI,TEMPERATURE_AMAX,TEMPERATURE_AMAX,TEMPERATURE_AMIN,DENGUE
3,REL_HUMIDITY_AMAX,SEMANA_lag_1,TEMPERATURE_AMIN,SST4,TEMPERATURE_AMIN,TEMPERATURE_AMIN,MEI,DRY_DAYS_lag_1
4,REL_HUMIDITY_AMIN,TEMPERATURE_RANGE_lag_1,TEMPERATURE_RANGE,TNI,REL_HUMIDITY_AMIN,REL_HUMIDITY_AMIN,SST1+2,DENGUE_lag_1
5,TEMPERATURE_RANGE,DENGUE_lag_1,SST4,DENGUE,TEMPERATURE_MEAN,REL_HUMIDITY_MEAN,SST3.4,ANO_lag_3
6,REL_HUMIDITY_MEAN,SEMANA_lag_2,DENGUE,ANO_lag_1,TEMPERATURE_RANGE,MEI,DENGUE,TEMPERATURE_AMAX_lag_4
7,SST1+2,REL_HUMIDITY_AMAX_lag_2,REL_HUMIDITY_AMAX_lag_1,SEMANA_lag_1,REL_HUMIDITY_MEAN,ONI,SEMANA_lag_1,TEMPERATURE_AMAX_lag_5
8,DENGUE,SST4_lag_2,TEMPERATURE_MEAN_lag_1,PRECIPITATION_lag_1,MEI,SST4,MEI_lag_1,TEMPERATURE_AMAX_lag_7
9,TEMPERATURE_RANGE_lag_1,TNI_lag_2,REL_HUMIDITY_MEAN_lag_1,TEMPERATURE_AMIN_lag_1,ONI,DENGUE,ONI_lag_1,TNI_lag_12


In [ ]:
best_features.to_csv('best_features_santa_marta.csv')